In [1]:
batch_size = 128
STROKE_COUNT = 196
TRAIN_SAMPLES = 1000
VALID_SAMPLES = 750
TEST_SAMPLES = 750

%matplotlib inline
import os
import numpy as np
import matplotlib.pyplot as plt
#from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
import pandas as pd
def top_3_accuracy(x,y): return top_k_categorical_accuracy(x,y, 3)
from glob import glob
import gc
gc.enable()

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import torch.nn as nn
import torch.optim as optim


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("./input"))
base_dir = os.path.join('./', 'input')
test_path = os.path.join(base_dir, 'test_simplified.csv')

# Any results you write to the current directory are saved as output.

['train_simplified', 'sample_submission.csv', 'train_raw.zip', 'test_raw.csv', 'test_simplified.csv', 'train_simplified.zip']


In [2]:
import six
def pad_sequences(sequences, maxlen=None, dtype='int32',
                  padding='pre', truncating='pre', value=0.):
    """Pads sequences to the same length.
    This function transforms a list of
    `num_samples` sequences (lists of integers)
    into a 2D Numpy array of shape `(num_samples, num_timesteps)`.
    `num_timesteps` is either the `maxlen` argument if provided,
    or the length of the longest sequence otherwise.
    Sequences that are shorter than `num_timesteps`
    are padded with `value` at the end.
    Sequences longer than `num_timesteps` are truncated
    so that they fit the desired length.
    The position where padding or truncation happens is determined by
    the arguments `padding` and `truncating`, respectively.
    Pre-padding is the default.
    # Arguments
        sequences: List of lists, where each element is a sequence.
        maxlen: Int, maximum length of all sequences.
        dtype: Type of the output sequences.
            To pad sequences with variable length strings, you can use `object`.
        padding: String, 'pre' or 'post':
            pad either before or after each sequence.
        truncating: String, 'pre' or 'post':
            remove values from sequences larger than
            `maxlen`, either at the beginning or at the end of the sequences.
        value: Float or String, padding value.
    # Returns
        x: Numpy array with shape `(len(sequences), maxlen)`
    # Raises
        ValueError: In case of invalid values for `truncating` or `padding`,
            or in case of invalid shape for a `sequences` entry.
    """
    if not hasattr(sequences, '__len__'):
        raise ValueError('`sequences` must be iterable.')
    lengths = []
    for x in sequences:
        if not hasattr(x, '__len__'):
            raise ValueError('`sequences` must be a list of iterables. '
                             'Found non-iterable: ' + str(x))
        lengths.append(len(x))

    num_samples = len(sequences)
    if maxlen is None:
        maxlen = np.max(lengths)

    # take the sample shape from the first non empty sequence
    # checking for consistency in the main loop below.
    sample_shape = tuple()
    for s in sequences:
        if len(s) > 0:
            sample_shape = np.asarray(s).shape[1:]
            break

    is_dtype_str = np.issubdtype(dtype, np.str_) or np.issubdtype(dtype, np.unicode_)
    if isinstance(value, six.string_types) and dtype != object and not is_dtype_str:
        raise ValueError("`dtype` {} is not compatible with `value`'s type: {}\n"
                         "You should set `dtype=object` for variable length strings."
                         .format(dtype, type(value)))

    x = np.full((num_samples, maxlen) + sample_shape, value, dtype=dtype)
    for idx, s in enumerate(sequences):
        if not len(s):
            continue  # empty list/array was found
        if truncating == 'pre':
            trunc = s[-maxlen:]
        elif truncating == 'post':
            trunc = s[:maxlen]
        else:
            raise ValueError('Truncating type "%s" '
                             'not understood' % truncating)

        # check `trunc` has expected shape
        trunc = np.asarray(trunc, dtype=dtype)
        if trunc.shape[1:] != sample_shape:
            raise ValueError('Shape of sample %s of sequence at position %s '
                             'is different from expected shape %s' %
                             (trunc.shape[1:], idx, sample_shape))

        if padding == 'post':
            x[idx, :len(trunc)] = trunc
        elif padding == 'pre':
            x[idx, -len(trunc):] = trunc
        else:
            raise ValueError('Padding type "%s" not understood' % padding)
    return x

from ast import literal_eval
ALL_TRAIN_PATHS = glob(os.path.join(base_dir, 'train_simplified', '*.csv'))
COL_NAMES = ['countrycode', 'drawing', 'key_id', 'recognized', 'timestamp', 'word']

def _stack_it(raw_strokes):
    """preprocess the string and make 
    a standard Nx3 stroke vector"""
    stroke_vec = literal_eval(raw_strokes) # string->list
    # unwrap the list
    in_strokes = [(xi,yi,i)  
     for i,(x,y) in enumerate(stroke_vec) 
     for xi,yi in zip(x,y)]
    c_strokes = np.stack(in_strokes)
    # replace stroke id with 1 for continue, 2 for new
    c_strokes[:,2] = [1]+np.diff(c_strokes[:,2]).tolist()
    c_strokes[:,2] += 1 # since 0 is no stroke
    # pad the strokes with zeros
    # print(c_strokes)
    return pad_sequences(c_strokes.swapaxes(0, 1), 
                         maxlen=STROKE_COUNT, 
                         padding='post').swapaxes(0, 1)
def read_batch(samples=5, start_row = 0):
    """
    load and process the csv files
    this function is horribly inefficient but simple
    """
    out_df_list = []
    for c_path in ALL_TRAIN_PATHS:
        c_df = pd.read_csv(c_path, nrows=samples, skiprows=start_row)
        #print(type(c_df), ";")
        c_df.columns=COL_NAMES
        out_df_list += [c_df[['drawing', 'word']]]
        #print(out_df_list, "==")
    full_df = pd.concat(out_df_list)
    full_df['drawing'] = full_df['drawing'].\
        map(_stack_it)
    
    return full_df

In [3]:
train_args = dict(samples=TRAIN_SAMPLES, 
                  start_row=0)
valid_args = dict(samples=VALID_SAMPLES, 
                  start_row=train_args['samples']+1)
test_args = dict(samples=TEST_SAMPLES, 
                 start_row=valid_args['samples']+train_args['samples']+1)
print("training set is loading")
train_df = read_batch(**train_args)
print("training set is loaded")
valid_df = read_batch(**valid_args)
print("dev set is loaded")
test_df = read_batch(**test_args)
print("test set is loaded")
word_encoder = LabelEncoder()
word_encoder.fit(train_df['word'])
print('words', len(word_encoder.classes_), '=>', ', '.join([x for x in word_encoder.classes_]))

training set is loading
training set is loaded
dev set is loaded
test set is loaded
words 340 => The Eiffel Tower, The Great Wall of China, The Mona Lisa, airplane, alarm clock, ambulance, angel, animal migration, ant, anvil, apple, arm, asparagus, axe, backpack, banana, bandage, barn, baseball, baseball bat, basket, basketball, bat, bathtub, beach, bear, beard, bed, bee, belt, bench, bicycle, binoculars, bird, birthday cake, blackberry, blueberry, book, boomerang, bottlecap, bowtie, bracelet, brain, bread, bridge, broccoli, broom, bucket, bulldozer, bus, bush, butterfly, cactus, cake, calculator, calendar, camel, camera, camouflage, campfire, candle, cannon, canoe, car, carrot, castle, cat, ceiling fan, cell phone, cello, chair, chandelier, church, circle, clarinet, clock, cloud, coffee cup, compass, computer, cookie, cooler, couch, cow, crab, crayon, crocodile, crown, cruise ship, cup, diamond, dishwasher, diving board, dog, dolphin, donut, door, dragon, dresser, drill, drums, duck, 

In [4]:
train_df = train_df.sample(frac=1).reset_index(drop=True)
#print("test_train_df", test_train_df)
#print("train_df", train_df)

In [5]:
def to_categorical(y, num_classes=None, dtype='float32'):
    """Converts a class vector (integers) to binary class matrix.
    E.g. for use with categorical_crossentropy.
    # Arguments
        y: class vector to be converted into a matrix
            (integers from 0 to num_classes).
        num_classes: total number of classes.
        dtype: The data type expected by the input, as a string
            (`float32`, `float64`, `int32`...)
    # Returns
        A binary matrix representation of the input. The classes axis
        is placed last.
    # Example
    ```python
    # Consider an array of 5 labels out of a set of 3 classes {0, 1, 2}:
    > labels
    array([0, 2, 1, 2, 0])
    # `to_categorical` converts this into a matrix with as many
    # columns as there are classes. The number of rows
    # stays the same.
    > to_categorical(labels)
    array([[ 1.,  0.,  0.],
           [ 0.,  0.,  1.],
           [ 0.,  1.,  0.],
           [ 0.,  0.,  1.],
           [ 1.,  0.,  0.]], dtype=float32)
    ```
    """

    y = np.array(y, dtype='int')
    input_shape = y.shape
    if input_shape and input_shape[-1] == 1 and len(input_shape) > 1:
        input_shape = tuple(input_shape[:-1])
    y = y.ravel()
    if not num_classes:
        num_classes = np.max(y) + 1
    n = y.shape[0]
    categorical = np.zeros((n, num_classes), dtype=dtype)
    categorical[np.arange(n), y] = 1
    output_shape = input_shape + (num_classes,)
    categorical = np.reshape(categorical, output_shape)
    return categorical
def get_Xy(in_df):
    X = np.stack(in_df['drawing'], 0)
    y = to_categorical(word_encoder.transform(in_df['word'].values))
    return X, y
train_X, train_y = get_Xy(train_df)
valid_X, valid_y = get_Xy(valid_df)
test_X, test_y = get_Xy(test_df)
print(train_X.shape)

(340000, 196, 3)


In [6]:
class HParams():
    def __init__(self):
        self.Nz = 128
        self.M = 20
        self.ConvKernelSize = 5
        self.dropout = 0.3
        self.batch_size = 100
        self.eta_min = 0.01
        self.R = 0.99995
        self.KL_min = 0.2
        self.wKL = 0.5
        self.lr = 0.001
        self.lr_decay = 0.9999
        self.min_lr = 0.00001
        self.grad_clip = 1.
        self.temperature = 0.4
        self.max_seq_length = 200

hp = HParams()


In [7]:
from torch.autograd import Variable
use_cuda = torch.cuda.is_available()
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
def make_batch_slice(start_index, batch_size):
    end_index = min(train_X.shape[0], start_index+batch_size)
    x_slice = train_X[start_index:end_index]
    y_slice = train_y[start_index:end_index]
    x_slice = x_slice.swapaxes(0,1)
    batch = Variable(torch.from_numpy(np.asarray(x_slice)).float()).to(device)
    ytags = Variable(torch.from_numpy(np.asarray(y_slice)).long()).to(device)
    #print("batch",batch)
    return batch, ytags
    

cuda


In [8]:
import torch.nn.functional as F
class SketchRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=1):
        super(SketchRNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        
        self.conv1d_1 = nn.Conv1d(input_size, 48, 5)
        self.dropout_1 = nn.Dropout(0.3)
        self.conv1d_2 = nn.Conv1d(48, 64, 5)
        self.dropout_2 = nn.Dropout(0.3)
        self.conv1d_3 = nn.Conv1d(64, 96, 3)
        self.dropout_3 = nn.Dropout(0.3)
        self.lstm_1 = nn.LSTM(96,hidden_size, n_layers, dropout=hp.dropout,batch_first=True)
        self.fc_mu = nn.Linear(hidden_size, output_size)
        

    def forward(self, inputs, hidden):
        inputs = inputs.transpose(0, 1).transpose(1, 2)
        #print("inputs",type(inputs))
        #print("inputs",inputs.size())
        output = self.conv1d_1(inputs)
        #print(output.size())
        output = self.dropout_1(output)
        output = self.conv1d_2(output)
        #print(output.size())
        output = self.dropout_2(output)
        output = self.conv1d_3(output)
        #print(output.size())
        output = self.dropout_3(output)
        output = output.transpose(1, 2)
        #output = output.transpose(1, 2).transpose(0, 1)
        #print("output",type(output))
        output, (hidden,x) = self.lstm_1(output, hidden)
        output_in_last_timestep=output[:,-1,:]
        #print("output",output.size())
        #print(hidden.size())
        #print(x.size())
        #print("output_in_last_timestep", output_in_last_timestep.size())
        output = self.fc_mu(output_in_last_timestep)
        #print("fc output", output.size())
        output = F.log_softmax(output, dim=1)
        return output, hidden 

model = SketchRNN(3, 128, 340).to(device)
optimizer = optim.Adam(model.parameters(),weight_decay = 1e-5,lr=0.1)

/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/nn/modules/rnn.py:38: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/cuda/__init__.py:116: UserWarning: 
    Found GPU0 GRID K520 which is of cuda capability 3.0.
    PyTorch no longer supports this GPU because it is too old.
    
  warnings.warn(old_gpu_warn % (d, name, major, capability[1]))


In [9]:

batch_size = 128
print_every = 2
for epoch in range(300):  # again, normally you would NOT do 300 epochs, it is toy data
    start_index = 0
    hidden = None
    while start_index < 10000:
        sentence, tags = make_batch_slice(start_index, batch_size)
        #print("sentence",type(sentence))
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 3. Run our forward pass.
        tag_scores, hidden = model(sentence, None)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        #print("tag_scores", tag_scores)
        #print("hidden", hidden.size())
        #print("tags", tags.size())
        loss = F.cross_entropy(tag_scores, torch.max(tags, 1)[1])
        loss.backward()
        optimizer.step()
        start_index = start_index + batch_size
        #print(start_index)
    #if epoch % print_every == 0:
    print('loss = %.4f ' % (loss.item()))


RuntimeError: cuda runtime error (48) : no kernel image is available for execution on the device at /opt/conda/conda-bld/pytorch_1532579245307/work/aten/src/THC/THCTensorCopy.cu:206